## Librerias usadas

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
import ast
import json
import gzip
import os
import warnings
from dateutil import parser
warnings.filterwarnings("ignore")
#Libreria para el analisis de sentimiento
from textblob import TextBlob

## Funcion para leer los archivos JSON

In [2]:
def leer(ruta):
    """Esta funcion lee los archivos JSON y los transforma a DataFrame"""
    lista=[]
    with open(ruta, 'r', encoding='UTF-8') as f:
        for line in f.readlines():
            try:
                data=ast.literal_eval(line)
                lista.append(data)
            except ValueError as e:
                lista.append(line)
                continue

    return pd.DataFrame(lista)

## Datos de Reviews

In [3]:
df_reviews=leer("../../dato/australian_user_reviews.json")
df_reviews

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [4]:
lista_review=[]
for ir, fr in df_reviews["reviews"].items():
    df_n_r=pd.json_normalize(fr)
    df_n_r["user_id"]=df_reviews.iloc[ir]["user_id"]
    lista_review.append(df_n_r)

In [5]:
df_reviews2=pd.concat(lista_review)
df_reviews2.head()

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479
1,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479
2,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
0,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637
1,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637


In [6]:
df_reviews2.drop(["helpful","last_edited","funny"],axis=1, inplace=True)
df_reviews2.head(3)

,posted,item_id,recommend,review,user_id
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479


In [7]:
#Cambiamos el formato de fecha
# Función para analizar la fecha y manejar errores
def parse_date(date_str):
    try:
        return parser.parse(date_str.replace("Posted ", ""), fuzzy=True)
    except ValueError:
        return None

# Aplicar la función de análisis de fecha y reemplazar filas con fechas incorrectas con NaN
df_reviews2['posted'] = df_reviews2['posted'].apply(parse_date)

# Eliminar filas con fechas incorrectas (NaN en la columna "posted")
df_reviews2 = df_reviews2.dropna(subset=['posted'])

# Ahora, la columna "posted" contiene las fechas en el formato deseado y se han eliminado las filas con fechas incorrectas


In [8]:
#Crear la funcion para el analisis de sentimiento
def sentimento(review):
    if isinstance(review, list) and len(review) > 0:
        text = review[0].get('review', '')  # Obtener el texto de la reseña
        sentiment = TextBlob(text).sentiment.polarity

        if sentiment < -0.2:
            return 0  # Malo
        elif sentiment >= -0.2 and sentiment <= 0.2:
            return 1  # Neutral
        else:
            return 2  # Positivo
    else:
        return 1  # Valor predeterminado para reseñas faltantes

In [9]:
# Aplicar la función a la columna 'reviews' y crear la nueva columna 'sentiment_analysis'
df_reviews2['sentiment_analysis'] = df_reviews['reviews'].apply(sentimento)

In [10]:
df_reviews2.head()

,posted,item_id,recommend,review,user_id,sentiment_analysis
0,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,1
1,2011-07-15,22200,True,It's unique and worth a playthrough.,76561197970982479,1
2,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2
0,2014-06-24,251610,True,I know what you think when you see this title ...,js41637,1
1,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,js41637,1


In [11]:
df_reviews2['item_id'] = df_reviews2['item_id'].fillna(0).astype(int)
df_reviews2['recommend'] = df_reviews2['recommend'].fillna(0).astype(bool)
df_reviews2['review'] = df_reviews2['review'].fillna(0).astype(str)
df_reviews2['user_id'] = df_reviews2['user_id'].fillna(0).astype(str)
df_reviews2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 59305 entries, 0 to 2
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   posted              59305 non-null  datetime64[ns]
 1   item_id             59305 non-null  int32         
 2   recommend           59305 non-null  bool          
 3   review              59305 non-null  object        
 4   user_id             59305 non-null  object        
 5   sentiment_analysis  59305 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int32(1), int64(1), object(2)
memory usage: 2.5+ MB


In [12]:
df_reviews2

,posted,item_id,recommend,review,user_id,sentiment_analysis
0,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,1
1,2011-07-15,22200,True,It's unique and worth a playthrough.,76561197970982479,1
2,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,2
0,2014-06-24,251610,True,I know what you think when you see this title ...,js41637,1
1,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,js41637,1
...,...,...,...,...,...,...
2,2024-07-10,70,True,a must have classic from steam definitely wort...,76561198312638244,2
3,2024-07-08,362890,True,this game is a perfect remake of the original ...,76561198312638244,2
0,2024-07-03,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,1
1,2024-07-20,730,True,:D,LydiaMorley,1


## Datos items

### Datos de la Columna "items"

- **item_id**: El identificador único del ítem.
- **item_name**: El nombre del ítem.
- **playtime_forever**: El tiempo total de juego (en minutos) del usuario para este ítem.
- **playtime_2weeks**: El tiempo de juego (en minutos) del usuario en las últimas dos semanas para este ítem.

"items" parece ser una lista de diccionarios en formato json, se procede a desarmar y agregar cada diccionario como columna al df principal

In [13]:
#Dado el tamaño del archivo JSON no se subira a git, y se convertira a formato parquet
df_items=leer("../../dato/australian_users_items.json")
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [14]:
#Extraer los datos guardados en los diccionarios de la columna de items
lista_items=[]
for i, f in df_items["items"].items():
    df_n_i=pd.json_normalize(f)
    df_n_i["user_id"]=df_items.iloc[i]["user_id"]
    #df_n_i["items_count"]=df_items.iloc[i]["items_count"]
    #df_n_i["steam_id"]=df_items.iloc[i]["steam_id"]
    lista_items.append(df_n_i)

In [15]:
df_items2=pd.concat(lista_items)
df_items2.head(5)

,item_id,item_name,playtime_forever,playtime_2weeks,user_id
0,10,Counter-Strike,6.0,0.0,76561197970982479
1,20,Team Fortress Classic,0.0,0.0,76561197970982479
2,30,Day of Defeat,7.0,0.0,76561197970982479
3,40,Deathmatch Classic,0.0,0.0,76561197970982479
4,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479


In [16]:
df_items2['item_id'] = df_items2['item_id'].fillna(0).astype(int)
df_items2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5153209 entries, 0 to 6
Data columns (total 5 columns):
 #   Column            Dtype  
---  ------            -----  
 0   item_id           int32  
 1   item_name         object 
 2   playtime_forever  float64
 3   playtime_2weeks   float64
 4   user_id           object 
dtypes: float64(2), int32(1), object(2)
memory usage: 216.2+ MB


## Datos games

In [17]:
df_games=pd.read_json("../../dato/output_steam_games.json", lines=True)
df_games.tail(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"
120444,None,None,Maze Run VR,None,http://store.steampowered.com/app/681550/Maze_...,None,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,1.0,681550.0,None


In [18]:
df_games.dropna(how="all", inplace=True)
df_games.head(3)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com


In [19]:
df_games.drop(["publisher","url","reviews_url","early_access"], axis=1,inplace=True)

In [20]:
df_games

,genres,app_name,title,release_date,tags,specs,price,id,developer
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",[Single-player],4.99,761140.0,Kotoshiro
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...","[Single-player, Multi-player, Online Multi-Pla...",Free To Play,643980.0,Secret Level SRL
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...","[Single-player, Multi-player, Online Multi-Pla...",Free to Play,670290.0,Poolians.com
88313,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",[Single-player],0.99,767400.0,彼岸领域
88314,None,Log Challenge,None,None,"[Action, Indie, Casual, Sports]","[Single-player, Full controller support, HTC V...",2.99,773570.0,None
...,...,...,...,...,...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]","[Single-player, Steam Achievements]",1.99,773640.0,"Nikita ""Ghost_RUS"""
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]","[Single-player, Steam Achievements, Steam Clou...",4.99,733530.0,Sacada
120442,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]","[Single-player, Steam Achievements, Steam Trad...",1.99,610660.0,Laush Dmitriy Sergeevich
120443,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...","[Single-player, Steam Achievements, Steam Cloud]",4.99,658870.0,"xropi,stev3ns"


In [23]:
df_games["release_date"]=df_games["release_date"].fillna(0)
df_games["release_date"]
# Aplicar la función de análisis de fecha y reemplazar filas con fechas incorrectas con NaN
df_games['release_date'] = df_games['release_date'].apply(fecha)

# Eliminar filas con fechas incorrectas (NaN en la columna "posted")
#df_games = df_games.dropna(subset=['release_date'])

# Ahora, la columna "posted" contiene las fechas en el formato deseado y se han eliminado las filas con fechas incorrectas
#df_games.info()

TypeError: Parser must be a string or character stream, not int

In [24]:
df_games['id'] = df_games['id'].fillna(0).astype(str)
#df_games['release_date'] = df_games['release_date'].astype('datetime64[ns]', format="%Y/%m/%d")
df_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   genres        28852 non-null  object
 1   app_name      32133 non-null  object
 2   title         30085 non-null  object
 3   release_date  32135 non-null  object
 4   tags          31972 non-null  object
 5   specs         31465 non-null  object
 6   price         30758 non-null  object
 7   id            32135 non-null  object
 8   developer     28836 non-null  object
dtypes: object(9)
memory usage: 2.5+ MB


## Guardar los datos es formato parquet para ahorrar espacio

In [25]:
df_items2.to_parquet("data/items.parquet", index=False)
df_reviews2.to_parquet("data/reviews.parquet", index=False)
#df_games.to_parquet("data/games.parquet", index=False)

## Leer los archivos formato parquet

In [26]:
items=pd.read_parquet("data/items.parquet")
reviews=pd.read_parquet("data/reviews.parquet")
games=df_games

## Analisis exploratorio de datos

In [27]:
games["release_date"]=pd.to_datetime(games["release_date"], errors="coerce")
games["Ano"]=games["release_date"].dt.year

In [121]:
items_mas_jugados=items.groupby("item_name")["playtime_forever"].sum().sort_values(ascending=False)
items_mas_jugados=pd.DataFrame({"item":list(items_mas_jugados.index),"Cantidad":items_mas_jugados.values})
items_mas_jugados=items_mas_jugados.head(20).sort_values(by="item")
alt.Chart(items_mas_jugados).mark_bar().encode(
    y=alt.Y("item:N").sort("-x"),
    x = "Cantidad")

alt.Chart(...)

In [117]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   posted              59305 non-null  datetime64[ns]
 1   item_id             59305 non-null  int32         
 2   recommend           59305 non-null  bool          
 3   review              59305 non-null  object        
 4   user_id             59305 non-null  object        
 5   sentiment_analysis  59305 non-null  int64         
dtypes: bool(1), datetime64[ns](1), int32(1), int64(1), object(2)
memory usage: 2.1+ MB


In [118]:
alt.Chart(reviews).mark_bar().encode(
    alt.X("sentiment_analysis:Q", bin=True), 
    y="count()"
    )

MaxRowsError: The number of rows in your dataset is greater than the maximum allowed (5000).

Try enabling the VegaFusion data transformer which raises this limit by pre-evaluating data
transformations in Python.
    >> import altair as alt
    >> alt.data_transformers.enable("vegafusion")

Or, see https://altair-viz.github.io/user_guide/large_datasets.html for additional information
on how to plot large datasets.

alt.Chart(...)

### 3) Top_3 desarrolladores con juegos mas recomendados

In [ ]:
reviews.groupby("item_id")["sentiment_analysis"].sum().sort_values(ascending=False)

item_id
440       4170
730       4150
4000      2011
570       1736
218620    1284
          ... 
288370       0
267380       0
378930       0
6950         0
436120       0
Name: sentiment_analysis, Length: 3682, dtype: int64